In [1]:
from torchHMM.model.discretized_HMM import DiscreteHMM, DISCRETIZATION_TECHNIQUES

In [2]:
import numpy as np
import pandas as pd
import pickle as pkl

In [3]:
raise ArithmeticError

ArithmeticError: 

In [4]:
from hmmlearn.hmm import CategoricalHMM, GaussianHMM
from gensim.models import KeyedVectors


In [5]:
with open('clickstream_experiment/data/preprocessed_data/ClickStream_test_cleaned.pkl', 'rb') as f:
    cs = pkl.load(f)

In [6]:
item_ids = np.unique([e.item_id for s in cs.sessions for e in s.event_list])
item_ids = {item_ids[i]: i for i in range(len(item_ids))}

Xs = [[item_ids[e.item_id] for e in s.event_list[:-1]] for s in cs.sessions]
l = [len(y) for y in Xs]

X = np.concatenate(Xs).reshape(-1, 1)

target = [item_ids[s.event_list[-1].item_id] for s in cs.sessions]

In [7]:
X

array([[147834],
       [ 75421],
       [126066],
       ...,
       [ 35192],
       [ 35192],
       [ 35192]])

In [8]:
chmm = CategoricalHMM(10, n_iter=10, verbose=True)


In [9]:
%time chmm.fit(X, l)

         1   -51584726.2270             +nan
         2   -47134596.2558    +4450129.9712
         3   -46900576.8503     +234019.4055
         4   -46676340.4289     +224236.4214
         5   -46446016.4979     +230323.9310
         6   -46199387.9653     +246628.5325
         7   -45935137.9591     +264250.0063
         8   -45658455.6387     +276682.3204
         9   -45382255.4581     +276200.1806
        10   -45123089.6758     +259165.7823


CPU times: user 15min 56s, sys: 3.39 s, total: 15min 59s
Wall time: 22min 21s


CategoricalHMM(n_components=10,
               random_state=RandomState(MT19937) at 0x7F81280B0A40,
               verbose=True)

In [10]:
l1 = chmm.score(np.array(X).reshape(-1, 1), l)
l2 = chmm.score(np.concatenate([[item_ids[e.item_id] for e in s.event_list] for s in cs.sessions]).reshape(-1, 1), np.array(l) + 1)
(l2 - l1) / len(l)

-inf

In [11]:
vectors = KeyedVectors.load(
    f"clickstream_experiment/data/preprocessed_data/vectors_train_100_10_5_8_cleaned.kv"
)
vecs = np.concatenate(
    [
        vectors.get_vector(k).reshape(1, -1)
        for k in list(vectors.key_to_index.keys())
    ]
)

In [12]:
unknown = vecs.mean(axis=0).reshape(1, -1)

def get_vec(e):
    try:
        return vectors.get_vector(e.item_id).reshape(1, -1)
    except:
        return unknown

In [13]:
Xs_c = [[get_vec(e) for e in s.event_list[:-1]] for s in cs.sessions]
l_c = [len(y) for y in X]

X_c = np.concatenate([a for as_ in Xs_c for a in as_], axis=0)

target_c = [get_vec(s.event_list[-1]) for s in cs.sessions]

In [14]:
X_c.shape

(4147529, 100)

In [15]:
w2vhmm = GaussianHMM(10, n_iter=10, verbose=True)

In [16]:
%time w2vhmm.fit(X_c, l_c)

/ziob/klaudia/miniconda3/envs/preproc/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
         1   827453863.6911             +nan
         2  1533789662.1799  +706335798.4887
         3  2542888564.9033 +1009098902.7235
         4  3571381039.1770 +1028492474.2736
         5  3571759098.6830     +378059.5061
         6  3572254106.2933     +495007.6103
         7  3572606887.6113     +352781.3180
         8  3572928327.8666     +321440.2553
         9  3573234042.9652     +305715.0986
        10  3573441342.5788     +207299.6135
Some rows of transmat_ have zero sum because no transition from the state was ever observed.


CPU times: user 1h 53min 44s, sys: 1min 10s, total: 1h 54min 55s
Wall time: 1h 52min 23s


GaussianHMM(n_components=10, verbose=True)

In [17]:
l1c = w2vhmm.score(X_c, l_c)
l2c = w2vhmm.score(np.concatenate([get_vec(e) for s in cs.sessions for e in s.event_list]), np.array(l_c) + 1)
(l2c - l1c) / len(l_c)

ValueError: transmat_ rows must sum to 1 (got [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.])

In [18]:
w2v_dhmm = DiscreteHMM(n_components=10, no_nodes=512, l=100, optimizer="Adam", verbose=True)

In [19]:
%time w2vhmm.fit(X_c, l_c, early_stopping=True)

TypeError: BaseHMM.fit() got an unexpected keyword argument 'early_stopping'

In [20]:
l1c_d = w2v_dhmm.score(X_c, l_c)
l2c_d = w2v_dhmm.score(np.concatenate([get_vec(e) for s in cs.sessions for e in s.event_list]), np.array(l_c) + 1)
(l2c_d - l1c_d) / len(l_c)

NotFittedError: This DiscreteHMM instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.